In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


### 문제 파악
- 타이타닉 우주선이 시공간 변칙성과 충돌했다.
- 그 안에 타고 있던 13000명의 승객 중 절반이 사고에 의하여 다른 차원으로 이송되었다.
- 손상된 컴퓨터 시스템에서 복구된 기록을 사용해 어떤 승객이 변칙적으로 운송되었는지 예측해야 한다.
- 내 임무는 우주선 타이타닉이 이 시공간 이상과 충돌하는 동안 승객이 다른 차원으로 이송되었는지 여부를 예측하는 것이다.

### 파일 및 데이터 필드
- train.csv - 훈련 데이터로 사용되는 승객의 약 2/3(~8700)에 대한 개인 기록.
   - PassengerId- 각 승객에 대한 고유 ID. 각 ID는 승객이 함께 여행하는 그룹 gggg_pp을 gggg나타내며 pp는 그룹 내 번호. 그룹의 사람들은 보통 가족 구성원이지만 항상 그런 것은 아니다.
   - HomePlanet- 승객이 출발한 행성, 일반적으로 영구 거주 행성.
   - CryoSleep- 승객이 항해 기간 동안 정지된 애니메이션에 놓이도록 선택했는지 여부를 나타낸다. cryosleep의 승객은 객실에 국한된다.
   - Cabin- 승객이 머물고 있는 객실 번호. Deck/num/side 형식을 취하며, 여기에서 side는 Port의 경우 P 또는 Starboard의 경우 S일 수 있다.
   - Destination- 목적지
   - Age- 승객의 나이.
   - VIP- 승객이 항해 중 특별 VIP 서비스를 지불했는지 여부.
   - RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - 승객이 우주선 타이타닉의 다양한 고급 편의 시설에 대해 청구한 금액.
   - Name- 승객의 성과 이름.
   - Transported- 승객이 다른 차원으로 이송되었는지 여부. 이것은 예측하려는 대상인 열.
- test.csv - 테스트 데이터로 사용되는 나머지 승객의 1/3(~4300)에 대한 개인 기록. Transported당신의 임무는 이 세트에 있는 승객의 가치를 예측하는 것.
- sample_submission.csv - 올바른 형식의 제출 파일.
 - PassengerId- 테스트 세트의 각 승객에 대한 ID.
 - Transported- 목표. 각 승객에 대해 True 또는 False를 예측.

# Step1. 라이브러리 및 데이터 불러오기
- 라이브러리 버전 확인

In [2]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas

clear_output()

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
print("numpy version is : ", np.__version__)
print("pandas version is : ", pd.__version__)
print("seaborn version is : ", sns.__version__)

numpy version is :  1.20.3
pandas version is :  1.3.5
seaborn version is :  0.11.2


In [5]:
# Data Loading and Preparation

train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

RANDOM_STATE = 12
FOLDS = 5
STRATEGY = 'median'

📌 기차 데이터의 관찰:
* 기차 데이터는 총 14개의 열과 8693개의 행이 있다.
* 기차 데이터에는 2324개의 결측값이 있는 119378개의 관측값이 포함되어 있다.
* 12개의 모든 기능 열에는 결측값이 있으며 CryoSleep에서는 결측값이 가장 높음(217)
* Transported는 기차 데이터 세트에서만 사용할 수 있는 대상 변수이다.

In [6]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.00,False,0.00,0.00,0.00,0.00,0.00,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.00,False,109.00,9.00,25.00,549.00,44.00,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.00,True,43.00,3576.00,0.00,6715.00,49.00,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.00,False,0.00,1283.00,371.00,3329.00,193.00,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.00,False,303.00,70.00,151.00,565.00,2.00,Willy Santantines,True


In [7]:
print(f'\033[94mNumber of rows in train data: {train.shape[0]}]')
print(f'\033[94mNumber of colunmns in train data: {train.shape[1]}]')
print(f'\033[94mNumber of values in train data: {train.count().sum()}]')
print(f'\033[94mNumber of missing values in train data: {sum(train.isna().sum())}]')

Number of rows in train data: 8693]
Number of colunmns in train data: 14]
Number of values in train data: 119378]
Number of missing values in train data: 2324]


- 열별 결측값

In [8]:
print(f'\033[94m')
print(train.isna().sum().sort_values(ascending = False))


CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64


- 다음은 개수, 평균, 표준편차, 최소값, 1사분위수, 중앙값, 3사분위수 및 최대값에 대한 정보를 포함하는 각 변수에 대한 기본 통계이다.

In [9]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.00,8512.00,8510.00,8485.00,8510.00,8505.00
mean,28.83,224.69,458.08,173.73,311.14,304.85
std,14.49,666.72,1611.49,604.70,1136.71,1145.72
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,19.00,0.00,0.00,0.00,0.00,0.00
50%,27.00,0.00,0.00,0.00,0.00,0.00
75%,38.00,47.00,76.00,27.00,59.00,46.00
max,79.00,14327.00,29813.00,23492.00,22408.00,24133.00


📌 테스트 데이터의 관찰:
* 테스트 데이터는 총 13열 4277행입니다.
* 기차 데이터에는 1117개의 결측값이 있는 54484개의 관측값이 포함되어 있습니다.
* 12개의 모든 기능 열에는 결측값이 있으며 FoodCourt는 결측값이 가장 높음(106)

In [10]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.00,False,0.00,0.00,0.00,0.00,0.00,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.00,False,0.00,9.00,0.00,2823.00,0.00,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.00,False,0.00,0.00,0.00,0.00,0.00,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.00,False,0.00,6652.00,0.00,181.00,585.00,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.00,False,10.00,0.00,635.00,0.00,0.00,Brence Harperez


In [11]:
print(f'\033[94mNumber of row in test data: {test.shape[0]}')
print(f'\033[94mNumber of columns in test data: {test.shape[1]}')
print(f'\033[94mNumber of values in train data: {test.count().sum()}')
print(f'\033[94mNumber of row with missing values in tset data: {sum(test.isna().sum())}')

Number of row in test data: 4277
Number of columns in test data: 13
Number of values in train data: 54484
Number of row with missing values in tset data: 1117


- 열별 결측값

In [12]:
print(f'\033[94')
print((test.isna().sum().sort_values(ascending = False)))

[94
FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
RoomService      82
VRDeck           80
PassengerId       0
dtype: int64


- 다음은 개수, 평균, 표준편차, 최소값, 1사분위수, 중앙값, 3사분위수 및 최대값에 대한 정보를 포함하는 각 변수에 대한 기본 통계이다.

In [13]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.00,4195.00,4171.00,4179.00,4176.00,4197.00
mean,28.66,219.27,439.48,177.30,303.05,310.71
std,14.18,607.01,1527.66,560.82,1117.19,1246.99
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,19.00,0.00,0.00,0.00,0.00,0.00
50%,26.00,0.00,0.00,0.00,0.00,0.00
75%,37.00,53.00,78.00,33.00,50.00,36.00
max,79.00,11567.00,25273.00,8292.00,19844.00,22272.00


- submission File

In [14]:
submission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


# Step2. 탐색적 자료 분석 (EDA)
- 데이터 시각화
- 산점도, 막대 그래프 등
- 그래프 해석해서 설명을 달아야 함
- 약간의 데이터 전처리

- 데이터 개요

In [15]:
train.drop(["PassengerId"], axis = 1, inplace = True)
test.drop(["PassengerId"], axis = 1, inplace = True)
TARGET = 'Transported'
FEATURES = [col for col in train.columns if col not in ['id', TARGET]]
RANDOM_STATE = 12

In [16]:
"""
train.iloc[:, :-1].describe().T.sort_values(by='std', ascending = False)\
.stype.background_gradient(cmap='GnBu')\
.bar(subset=["max"], color='#BB0000')\
.bar(subset=["mean",], color='green')
"""

'\ntrain.iloc[:, :-1].describe().T.sort_values(by=\'std\', ascending = False).stype.background_gradient(cmap=\'GnBu\').bar(subset=["max"], color=\'#BB0000\').bar(subset=["mean",], color=\'green\')\n'

# Step3. 데이터 전처리
- Feature Engineering 이라고도 한다
- ML 모형을 돌리기 위해 표준화 등 / 원핫-인코딩
- 파생변수 (도출 변수) 만들기
    + 왜 이 변수를 만들었는지에 대한 본인의 설명 필요

# Step4. 머신러닝 모형 개발
- 모형에 대한 설명
- 모형을 1-2개 사용
- 교차 검증
- 하이퍼파라미터 튜닝

# Step5. 
- 훈련데이터 쪼갠다. 훈련데이터 + 검증데이터로 분리
- 정확도 비교
- 혼동행렬(Confusion Matriax) 설명 -> 스스로 찾아보고 설명을 넣는다

# Step6. 제출
- 제출 양식은 샘플 만들어지는거 보고 참고

# Reference
- 다른 사람의 code 설명을 따라친다면 해야 함.
- 노트북 표절 방지 위해, 본인이 참조한 코드는 반드시 링크 걸어둘 것.
- [저자 이름, 글 제목, 링크 주소]
- SANSKAR HASIJA, 🚀Spaceship Titanic -📊EDA + 27 different models📈, https://www.kaggle.com/code/odins0n/spaceship-titanic-eda-27-different-models



# 마감일
- 4월 12일 17시 40분
- 제출 형태
    + Leaderbord 랭킹 사진 캡처
    + 고용노동부 보고 양식(다음주에 확인해서 공지예정)